# Explore here

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from pickle import dump
import os
import opendatasets as od
import zipfile
import tensorflow as tf
from keras.preprocessing import image
from pathlib import Path
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.optimizers import Adam, SGD
import keras_tuner as kt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.losses import *
from tensorflow.keras import *

In [ ]:
# Assign the Kaggle data set URL into variable
dataset = "https://www.kaggle.com/c/dogs-vs-cats/data/dogs-vs-cats"
# Using opendatasets let's download the data sets
od.download(dataset, data_dir="../data/raw")

In [ ]:
with zipfile.ZipFile("../data/raw/dogs-vs-cats/test1.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/raw/dogs-vs-cats/")

In [ ]:
with zipfile.ZipFile("../data/raw/dogs-vs-cats/train.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/raw/dogs-vs-cats/")

In [ ]:
os.makedirs("../data/raw/dogs-vs-cats/train/dog")
os.makedirs("../data/raw/dogs-vs-cats/train/cat")

In [ ]:
for im in Path("./data/raw/dogs-vs-cats/train").glob('dog*.jpg'):
    shutil.move(im, os.path.join("../data/raw/dogs-vs-cats/train/dog", im.name))

for im in Path("./data/raw/dogs-vs-cats/train").glob('cat*.jpg'):
    shutil.move(im, os.path.join("../data/raw/dogs-vs-cats/train/cat", im.name))

In [ ]:
for im in [x for x in os.listdir("../data/raw/dogs-vs-cats/train/dog")][:9]:
    i = 0
    img = image.load_img(os.path.join("../data/raw/dogs-vs-cats/train/dog", im))
    img_array = image.img_to_array(img)
    img_array /= 255.0  # Normalice los valores de los píxeles a [0, 1].
    plt.subplot(3, 3, i + 1)
    plt.imshow(img_array)
    plt.title('Dog Image')
    plt.axis('off')
    i+=1

    plt.tight_layout()
    plt.show()

In [ ]:
for im in [x for x in os.listdir("../data/raw/dogs-vs-cats/train/cat")][:9]:
    i = 0
    img = image.load_img(os.path.join("../data/raw/dogs-vs-cats/train/cat", im))
    img_array = image.img_to_array(img)
    img_array /= 255.0  # Normalice los valores de los píxeles a [0, 1].
    plt.subplot(3, 3, i + 1)
    plt.imshow(img_array)
    plt.title('Cat Image')
    plt.axis('off')
    i+=1

    plt.tight_layout()
    plt.show()

In [ ]:
len(os.listdir("../data/raw/dogs-vs-cats/train/dog"))

In [ ]:
len(os.listdir("../data/raw/dogs-vs-cats/train/cat"))

El dataset está balanceado.

In [ ]:
train_format = ImageDataGenerator(rescale=1.0/255, validation_split=0.20)
test_format = ImageDataGenerator(rescale=1.0/255)

# Generación de datos de entrenamiento
train_data = train_format.flow_from_directory(
    "../data/raw/dogs-vs-cats/train",
    subset='training',
    target_size = (200, 200),
    color_mode='rgb',
    class_mode='binary',
    classes = ["dog", "cat"]
)

# Generación de datos de validación
val_data = train_format.flow_from_directory(
    "../data/raw/dogs-vs-cats/train",
    subset='validation',
    target_size = (200, 200),
    color_mode='rgb',
    class_mode='binary',
    classes = ["dog", "cat"]
)

# Generación de datos de prueba
test_data = test_format.flow_from_directory(
    "../data/raw/dogs-vs-cats/train",
    target_size = (200, 200),
    color_mode='rgb',
    class_mode='binary',
    classes = ["test1"]
)

In [ ]:
'''model_b = Sequential()
model_b.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
model_b.add(MaxPooling2D((2, 2)))
model_b.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_b.add(MaxPooling2D((2, 2)))
model_b.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_b.add(MaxPooling2D((2, 2)))
model_b.add(Flatten())
model_b.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_b.add(Dense(1, activation='sigmoid'))

# compile model
opt = SGD(learning_rate=0.001, momentum=0.9)
model_b.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

hist = model_b.fit(x=train_data, steps_per_epoch = 10, validation_data = val_data, validation_steps = 10, epochs = 10)'''

In [ ]:
model_b = Sequential()
model_b.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
model_b.add(MaxPooling2D((2, 2)))
model_b.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_b.add(MaxPooling2D((2, 2)))
model_b.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_b.add(MaxPooling2D((2, 2)))
model_b.add(Flatten())
model_b.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_b.add(Dense(1, activation='sigmoid'))

# compile model
opt = SGD(learning_rate=0.001, momentum=0.9)
model_b.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

hist = model_b.fit(x=train_data, batch_size = 64, validation_data = val_data, validation_batch_size = 64, epochs = 10)

In [ ]:
model_b.save('/content/gdrive/MyDrive/Colab Notebooks/model_short_b64_e10.keras')

In [ ]:
loaded_model = load_model('/content/gdrive/MyDrive/Colab Notebooks/model_short_b64_e10.keras')

In [ ]:
checkpoint = ModelCheckpoint('/content/gdrive/MyDrive/Colab Notebooks/model_short_b64_e10.keras', monitor = "val_accuracy", verbose = 1, save_best_only = True, save_weights_only = False, mode = "auto")
early = EarlyStopping(monitor = "val_accuracy", patience = 3, verbose = 1, mode = "auto")
hist = loaded_model.fit(x=train_data, batch_size = 64, validation_data = val_data, validation_batch_size = 64, epochs = 10, callbacks = [checkpoint, early])

In [ ]:
loaded_model.save('/content/gdrive/MyDrive/Colab Notebooks/model_short_b64_e10_with_cp.keras')

In [ ]:
# Comprobar la exactitud de los datos

# Trazar los resultados
plt.plot(hist.history["accuracy"])
plt.plot(hist.history["val_accuracy"])
plt.plot(hist.history["loss"])
plt.plot(hist.history["val_loss"])

# Configurar el diseño del gráfico
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy", "Validation Accuracy", "Loss", "Validation Loss"])

# Trazar
plt.show()

In [ ]:
# Comprobamos el modelo

from keras.preprocessing import image
from keras.models import load_model

img = image.load_img("../data/raw/dogs-vs-cats/test1/1218.jpg", target_size = (200, 200))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis = 0)
#saved_model = load_model("/content/gdrive/MyDrive/Colab Notebooks/model_short.keras")
output = loaded_model.predict(img)
if output[0][0] < 0.5:
    print("dog")
else:
    print("cat")

Visualizando algunas de las imágenes con el predictor, observo que casos de perros con patrón blanco y negro suelen ser identificados como gatos (ya que es un patrón más común en gatos que en perros).

Tras añadir checkpoints, ninguno de los dos se ejecuta, por lo que el modelo inicial se mantiene.
Dejo comentados otro modelo, y el final pero con otros hiperparámetros comentados, como referencia, pero han resultado bastante peor en resultados según las gráficas.